In [ ]:
!python --version

In [ ]:
!pip install swig
!pip install Boost
!pip install xylib-py
# !git clone https://github.com/wojdyr/xylib.git
# !pip install git+https://github.com/wojdyr/xylib.git
# !python setup.py install

!pip install swig Boost 
export PATH=$PATH:
!pip install xylib-py

In [ ]:
import xylib as xy

import glob, os 

#data management
import numpy as np, pandas as pd

#plotting
import matplotlib.pyplot as plt

xylib is a library for reading files that contain x-y data from powder diffraction, spectroscopy or other experimental methods.
It is recommended to set LC_NUMERIC="C" (or other locale with the same numeric format) before reading files.  Usually, we first call load_file() to read file from disk. It stores all data from the file in class DataSet.
  DataSet contains a list of Blocks, each Blocks contains a list of Columns,
  and each Column contains a list of values.
 
  It may sound complex, but IMO it can't be made simpler.
  It's analogical to a spreadsheet. One OOCalc or Excel file (which
  corresponds to xylib::DataSet) contains a number of sheets (Blocks),
  but usually only one is used. Each sheet can be viewed as a list of columns.
 
  In xylib all columns in one block must have equal length.
  Several filetypes always contain only one Block with two Columns.
  In this case we can take coordinates of the 15th point as:
     double x = get_block(0)->get_column(1)->get_value(14);
     double y = get_block(0)->get_column(2)->get_value(14);
  Note that blocks and points are numbered from 0, but columns are numbered
  from 1, because the column 0 returns index of point.
  All values are stored as floating-point numbers, even if they are integers
  in the file.
  DataSet and Block contain also MetaData, which is a string to string map.
 
  Note that C++ API uses std::string and exceptions, so it is recommended
  to compile the library and programs that use it with the same compiler.
 
  C++ API is defined in xylib namespace, C API use prefix xylib.
 /

Read file and get block and the two columns

In [ ]:
filename = 'FILENAME.raw'
file1= xy.load_file('./' + filename)

print(file1.get_block_count())
block = file1.get_block(0)
print(block.get_column_count())
col1 = block.get_column(1)
col2 = block.get_column(2)

Get metadata from which u can obtain the x axis (seems that col1 is not accessible

In [ ]:
print(block.get_name())
meta = block.meta
type(meta)

keys = []

#get the keys name
for i in range(0,12):
  keys.append(meta.get_key(i))

#loop over keys to get their values
for key in keys:
  print(key ,meta.get(key))
startth = meta.get('START_2THETA')
stepsize = meta.get('STEP_SIZE')
nstep = meta.get('STEPS')

print(type(startth), type(stepsize), type(nstep))
print(float(startth), float(stepsize), float(nstep))

startth = float(startth)
stepsize = float(stepsize)
nstep = float(nstep)

# creat np array for x
npx = np.arange(startth,startth+stepsize*nstep,stepsize)
np.shape(npx)

Extract y values

In [ ]:
valy = []

for index in range(1, col2.get_point_count()):
  valy.append(col2.get_value(index))

# create numpy array
npy = np.array(valy)

#build the dataset
dataset = [npx[:-1], npy]

npdataset = np.array(dataset).T
np.shape(npdataset)

construct the head of the final file

In [ ]:
tps = meta.get('TIME_PER_STEP')
lam = meta.get('USED_LAMBDA')
head = f"time per step: {tps}\nlambda: {lam}\n2theta Counts"

In [ ]:
# plot dataset
plt.plot(dataset[0], dataset[1])

Export and save

In [ ]:
np.savetxt(fname=filename[:-3]+".xy", X=npdataset, header=head)

The following cell is to run the same code on multiple files.

In [ ]:
########################### MODIFY THESE PRMS ###########################
#path parameter
mainDir = './'
#leave empty(with no '' !!) if not used, 
subDirs = []

#fill this list to run the code only for specified folders. 
selected = []    #REMEMBER to put extension. 

#file parameter
ext = '.raw'
#############################################################################

#variable declaration
paths = []
files = []  #is a list of filenames choosen
dataset = []  #is a list of np.array rapresenting the whole dataset

#Generate the complete paths for each defined subdir
for subDir in subDirs:
  paths.append(mainDir+subDir)

print(paths)

#import files in selected 
if len(selected) == 0:
  if len(subDirs) != 0:
    selected = glob.glob(mainDir+'**/*'+ext, recursive=True)   # if filenames is empty, take all folders/files inside maindir
  else:
    selected = glob.glob(mainDir+'*'+ext, recursive=True)

#sort selected files
selected.sort()
print(len(selected), selected)


# create and clean export folder
if os.path.exists('./export/'): 
  !rm -r export/ 
  !mkdir export
else: 
  !mkdir export


for path in selected:
  file = xy.load_file(path)
  #usually raws are made of 1 block
  block = file.get_block(0)

  #extract the two columns, even though seems that col1 doesn't contain 2theta values
  col1 = block.get_column(1)
  col2 = block.get_column(2)
  print(f"file: {file}, n blocks: {file.get_block_count()}, n cols in block: {block.get_column_count()}")

  #extract meta information from which we can build-up 2theeta axis.
  meta = block.meta
  keys = []   #container for keys

  #get the keys' name
  for i in range(0,12):
    keys.append(meta.get_key(i))

  #loop over keys to get their values
  for key in keys:
    print(key ,meta.get(key))
  startth = float(meta.get('START_2THETA'))
  stepsize = float(meta.get('STEP_SIZE'))
  nstep = float(meta.get('STEPS'))
  tps = float(meta.get('TIME_PER_STEP'))
  lam = float(meta.get('USED_LAMBDA'))

  # create np array for x
  npx = np.arange(startth,startth+stepsize*nstep,stepsize)
  print(f"{np.shape(npx)} points on x axis" )

  # extract y values
  valy = []
  for index in range(1, col2.get_point_count()):
    valy.append(col2.get_value(index))

  # create numpy array
  npy = np.array(valy)
  print(f"{np.shape(npy)} points on y axis" )
  npnorm = npy/tps

  #build the dataset
  if np.shape(npy) != np.shape(npx):
    dataset = [npx[:-1], npy, npnorm]   #usually x points are 1 time longer.
  else:
    dataset = [npx, npy, npnorm]

  #generate np.array
  npdataset = np.array(dataset).T

  #build the head of the final .xy files
  head = f"time per step: {tps}\nlambda: {lam}\n2theta Counts NormalizedCounts"


  if not os.path.exists('./export'+path):
    !mkdir {'./export'+path.strip(os.path.basename(path))}

  # save array to .xy file
  np.savetxt(fname='./export/'+path[2:-4]+".xy", X=npdataset[:,:2], header=head)
  np.savetxt(fname='./export/'+path[2:-4]+".xyn", X=npdataset[:,:2], header=head)

  !zip -r export.zip export